In [1]:
import numpy
import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import  LSTM as LSTM, SimpleRNN, CuDNNGRU as GRU
#from keras.layers import LSTM , SimpleRNN, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [2]:
import pandas as pd
data = pd.read_csv("smsspamcollection/SMSSpamCollection.txt", header = None, sep="\t")

In [3]:
data.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data.columns = ['label','text']

In [5]:
data['y']=0

In [6]:
data.loc[data['label']=='spam','y']=1

In [7]:
data['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [8]:
import collections
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [9]:
word_list=[]
for i in data['text']:
    word_list = word_list + i.split()

In [10]:
word_list

['Go',
 'until',
 'jurong',
 'point,',
 'crazy..',
 'Available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet...',
 'Cine',
 'there',
 'got',
 'amore',
 'wat...',
 'Ok',
 'lar...',
 'Joking',
 'wif',
 'u',
 'oni...',
 'Free',
 'entry',
 'in',
 '2',
 'a',
 'wkly',
 'comp',
 'to',
 'win',
 'FA',
 'Cup',
 'final',
 'tkts',
 '21st',
 'May',
 '2005.',
 'Text',
 'FA',
 'to',
 '87121',
 'to',
 'receive',
 'entry',
 'question(std',
 'txt',
 "rate)T&C's",
 'apply',
 "08452810075over18's",
 'U',
 'dun',
 'say',
 'so',
 'early',
 'hor...',
 'U',
 'c',
 'already',
 'then',
 'say...',
 'Nah',
 'I',
 "don't",
 'think',
 'he',
 'goes',
 'to',
 'usf,',
 'he',
 'lives',
 'around',
 'here',
 'though',
 'FreeMsg',
 'Hey',
 'there',
 'darling',
 "it's",
 'been',
 '3',
 "week's",
 'now',
 'and',
 'no',
 'word',
 'back!',
 "I'd",
 'like',
 'some',
 'fun',
 'you',
 'up',
 'for',
 'it',
 'still?',
 'Tb',
 'ok!',
 'XxX',
 'std',
 'chgs',
 'to',
 'send,',
 '£1.50',
 'to',
 'rcv',
 '

In [11]:
enc, dec = build_dataset(word_list)

In [12]:
for i in enc:
    enc[i] = enc[i]+2
    
enc['pad']=0
enc['<start>']=1
dec[-2]='<pad>'
dec[-1]='<start>'

In [13]:
import numpy as np
n=int(np.floor(data.shape[0]*0.75))
train = data[0:n]
test = data[n:]

In [14]:
x_train=[]
y_train=[]
for i in range(train.shape[0]):
    tmp = [enc[j] for j in train.iloc[i,1].split()]
    x_train.append(tmp)
    if train.iloc[i,0]=='spam':
        y=1
    else:
        y=0
    y_train.append(y)
    
x_test=[]
y_test=[]
for i in range(test.shape[0]):
    tmp = [enc[j] for j in test.iloc[i,1].split()]
    x_test.append(tmp)
    if test.iloc[i,0]=='spam':
        y=1
    else:
        y=0
    y_test.append(y)

In [15]:
[dec[i-2] for i in x_train[0]]

['Go',
 'until',
 'jurong',
 'point,',
 'crazy..',
 'Available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet...',
 'Cine',
 'there',
 'got',
 'amore',
 'wat...']

In [16]:
import matplotlib.pyplot as plt
plt.hist([len(i) for i in x_train])
plt.show()

<Figure size 640x480 with 1 Axes>

In [17]:
len(enc)

15693

In [18]:
max_length = 50
x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_length)

In [19]:
embedding_vecor_length = 80
model1 = Sequential()
model1.add(Embedding(15693, embedding_vecor_length, input_length=max_length))
model1.add(SimpleRNN(100,unroll=True))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())
model1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=64)

W1010 20:14:45.721573  5392 deprecation.py:506] From c:\users\rdsla\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1010 20:14:45.737577  5392 deprecation.py:506] From c:\users\rdsla\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1010 20:14:45.976644  5392 deprecation.py:323] From c:\users\rdsla\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\nn_impl.py

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 80)            1255440   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               18100     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 1,273,641
Trainable params: 1,273,641
Non-trainable params: 0
_________________________________________________________________
None
Train on 4179 samples, validate on 1393 samples
Epoch 1/3
4179/4179 [==============================] - 2s 536us/sample - loss: 0.3935 - acc: 0.8636 - val_loss: 0.3473 - val_acc: 0.8693
Epoch 2/3
4179/4179 [==============================] - 1s 303us/sample - loss: 0.1216 - acc: 0.9627 - val_loss: 0.0676 - val_acc: 0.9792
Epoch 3/3
4179/4179 [============================

In [54]:
from tensorflow.keras.layers import Bidirectional, TimeDistributed
embedding_vecor_length = 100
model5 = Sequential()
model5.add(Embedding(15693, embedding_vecor_length, input_length=max_length))
model5.add((Bidirectional(SimpleRNN(100,return_sequences=False, dropout=0.5))))
model5.add(Dense(1, activation='sigmoid'))
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model5.summary())
model5.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 50, 100)           1569300   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1000)              2404000   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1001      
Total params: 3,974,301
Trainable params: 3,974,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 4179 samples, validate on 1393 samples
Epoch 1/5
4179/4179 [==============================] - 15s 4ms/sample - loss: 0.3045 - acc: 0.8899 - val_loss: 0.0875 - val_acc: 0.9742
Epoch 2/5
4179/4179 [==============================] - 13s 3ms/sample - loss: 0.0555 - acc: 0.9847 - val_loss: 0.0532 - val_acc: 0.9835
Epoch 3/5
4179/4179 [===========================